In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential, Model, load_model
from keras import backend as K

import tensorflow as tf

import os
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import aparent.visualization as vis

from aparent.predictor import *

import urllib
import urllib.request
import pickle
from time import sleep


Using TensorFlow backend.


In [2]:
#Load APADB Data and filter on targeted genes

#genes = ['RUNX1', 'CEBPA', 'GATA2', 'ANKRD26', 'DDX41', 'ETV6', 'PTEN', 'BRCA1', 'BRCA2', 'TP53', 'APC', 'ATM', 'PALB2', 'MSH2', 'MLH1', 'MSH6', 'PMS2', 'MUTYH']

polyadb_df = pd.read_csv('../../../data/native_data/polyadb/processed_data/polyadb_processed.csv', sep=',')

#polyadb_df = polyadb_df.loc[polyadb_df['gene'].isin(genes)].reset_index(drop=True).copy()
polyadb_df = polyadb_df.loc[~polyadb_df['gene'].isna()].reset_index(drop=True).copy()
polyadb_df = polyadb_df.loc[polyadb_df['gene'] != 'na'].reset_index(drop=True).copy()

print('len(polyadb_df) = ' + str(len(polyadb_df)))


len(polyadb_df) = 190117


In [3]:
polyadb_df_minus = polyadb_df.query("strand == '-'").copy().reset_index(drop=True)
polyadb_df_plus = polyadb_df.query("strand == '+'").copy().reset_index(drop=True)

polyadb_df_minus = polyadb_df_minus.sort_values(by='pas_pos', ascending=False).copy().reset_index(drop=True)
polyadb_df_plus = polyadb_df_plus.sort_values(by='pas_pos', ascending=True).copy().reset_index(drop=True)

new_gene_id_list_plus = []
sitenum_list_plus = []
gene_id_dict = {}
for _, row in polyadb_df_plus.iterrows() :

    gene = str(row['gene'])

    if gene not in gene_id_dict :
        gene_id_dict[gene] = 0

    gene_id_dict[gene] += 1

    new_gene_id_list_plus.append(gene + "." + str(gene_id_dict[gene]))
    sitenum_list_plus.append(gene_id_dict[gene])

polyadb_df_plus['gene_id'] = new_gene_id_list_plus
polyadb_df_plus['sitenum'] = sitenum_list_plus

new_gene_id_list_minus = []
sitenum_list_minus = []
gene_id_dict = {}
for _, row in polyadb_df_minus.iterrows() :

    gene = str(row['gene'])

    if gene not in gene_id_dict :
        gene_id_dict[gene] = 0

    gene_id_dict[gene] += 1

    new_gene_id_list_minus.append(gene + "." + str(gene_id_dict[gene]))
    sitenum_list_minus.append(gene_id_dict[gene])

polyadb_df_minus['gene_id'] = new_gene_id_list_minus
polyadb_df_minus['sitenum'] = sitenum_list_minus

polyadb_df = pd.concat([polyadb_df_plus, polyadb_df_minus])

polyadb_df = polyadb_df.sort_values(by=['gene', 'sitenum'], ascending=True).reset_index(drop=True).copy()

In [4]:
polyadb_df.head()

,gene,gene_id,sitenum,num_sites,pas,seq,seq_ext,wide_seq,wide_seq_ext,rpm,total_rpm,ratio,pas_pos,cut_mode,chrom,strand,site_type
0,A1BG,A1BG.1,1,11,0,TTCTGCTAGGCTGGGCTAGTATCTTCCATGGCAAGATACTCAAACT...,TAGGGAATATAACTTTTAAAATGCGAAAGTATATTATACATATTGC...,AGTTTGGCTAGCATTTCAAAATGGATAGGGAATATAACTTTTAAAA...,GCTTTCAAAATTCCTGATTAGCTTACAATTAATTGCAAATAACTTC...,1.564234,24.390222,0.064134,58859852,58859830,chr19,-,Intron
1,A1BG,A1BG.2,2,11,-1,AGGTGCCGCTACCGCTCCTGGGTGCCCCACACCTTCGAATCGGAGC...,CGGCGAACCTCGAGCTGATCTTCGTGGGGCCCCAGCACGCCGGCAA...,AAGACGGTCCGCACCCCCGGGGCCGCGGCGAACCTCGAGCTGATCT...,TCCCCGACGTCACCTTCGAGCTGCTGCGCGAGGGCGAGACGAAGGC...,1.740036,24.390222,0.071342,58858744,58858719,chr19,-,Internal_exon
2,A1BG,A1BG.3,3,11,0,CCGGGGTCTCTGGGGGGCTGGAGAAGCCTCCCTCATTCCTCCCAGG...,AGTGCCGGGGATTCTGGACTGGCTCCCTCCCCTCCTGTTGCAGCAC...,CCAGGGTGCTGTTGGTGTCCTCAGAAGTGCCGGGGATTCTGGACTG...,CACCATTTCTCTTTTCTCCCGTTGTCTAGAAAGCTGATGCAGCCGC...,11.529986,24.390222,0.472730,58858249,58858220,chr19,-,3_most_exon
3,A1BG,A1BG.4,4,11,4,GAAGATGCGTATCAGTATTATCAACCAATAGTTGATGAATTGTGTT...,ACCACTGCCGAGTTTCTGCCTGGAGCAGGGCCTCAAGGCCCTGGCA...,CCATCTGACATCTGATGCCTTCCCAACCACTGCCGAGTTTCTGCCT...,GATTCCTCTGTGTTCTTTCCCATTATCCCTCAGCTCTGAGTCCTCT...,1.386686,24.390222,0.056854,58857215,58857210,chr19,-,3_most_exon
4,A1BG,A1BG.5,5,11,4,AACGAATTTGCTAGTGATCTGGTTTACTGCCTTAGTAATATCTAGT...,GAAGATGCGTATCAGTATTATCAACCAATAGTTGATGAATTGTGTT...,CAGGGCCTCAAGGCCCTGGCACACAGAAGATGCGTATCAGTATTAT...,CCATCTGACATCTGATGCCTTCCCAACCACTGCCGAGTTTCTGCCT...,1.174863,24.390222,0.048169,58857165,58857142,chr19,-,3_most_exon


In [5]:

polyadb_df['start'] = polyadb_df['pas_pos'] - 150
polyadb_df['end'] = polyadb_df['pas_pos'] + 150

polyadb_bed = polyadb_df[['chrom', 'start', 'end', 'gene', 'gene_id', 'strand']].copy().reset_index(drop=True)

polyadb_bed.head()

polyadb_bed.to_csv("polyadb_coordinates.bed", sep='\t', header=False, index=False)


In [6]:
#Tissue types to compile data for

tissue_types = [
    'Adipose_Subcutaneous',
    'Adipose_Visceral_Omentum',
    'Adrenal_Gland',
    'Artery_Aorta',
    'Artery_Coronary',
    'Artery_Tibial',
    'Brain_Amygdala',
    'Brain_Anterior_cingulate_cortex_BA24',
    'Brain_Caudate_basal_ganglia',
    'Brain_Cerebellar_Hemisphere',
    'Brain_Cerebellum',
    'Brain_Cortex',
    'Brain_Frontal_Cortex_BA9',
    'Brain_Hippocampus',
    'Brain_Hypothalamus',
    'Brain_Nucleus_accumbens_basal_ganglia',
    'Brain_Putamen_basal_ganglia',
    'Brain_Spinal_cord_cervical_c-1',
    'Brain_Substantia_nigra',
    'Breast_Mammary_Tissue',
    'Cells_EBV-transformed_lymphocytes',
    'Cells_Cultured_fibroblasts',
    'Colon_Sigmoid',
    'Colon_Transverse',
    'Esophagus_Gastroesophageal_Junction',
    'Esophagus_Mucosa',
    'Esophagus_Muscularis',
    'Heart_Atrial_Appendage',
    'Heart_Left_Ventricle',
    'Kidney_Cortex',
    'Liver',
    'Lung',
    'Minor_Salivary_Gland',
    'Muscle_Skeletal',
    'Nerve_Tibial',
    'Ovary',
    'Pancreas',
    'Pituitary',
    'Prostate',
    'Skin_Not_Sun_Exposed_Suprapubic',
    'Skin_Sun_Exposed_Lower_leg',
    'Small_Intestine_Terminal_Ileum',
    'Spleen',
    'Stomach',
    'Testis',
    'Thyroid',
    'Uterus',
    'Vagina',
    'Whole_Blood'
]


In [12]:
#Convert to hg19 coordinates (for compatability with polyadb)

batch_size = 500

rs_dict = {}

for tissue_type in tissue_types :
    
    print("Processing " + tissue_type + "...")
    
    tissue_df = pd.read_csv(tissue_type + ".3aQTL.txt", sep='\t')
    
    rs_ids_w_prefix = tissue_df['rs'].values.tolist()
    rs_ids = np.unique([rs[2:] for rs in rs_ids_w_prefix if rs not in rs_dict]).tolist()
    
    n_batches = len(rs_ids) // batch_size + (1 if len(rs_ids) % batch_size > 0 else 0)

    print("- n_batches = " + str(n_batches))
    
    for batch_ix in range(0, n_batches) :

        print("Processing rs record batch " + str(batch_ix) + "...")

        rs_id_batch = rs_ids[batch_ix * batch_size: (batch_ix + 1) * batch_size]
        rs_id_str = ",".join(rs_id_batch)

        !epost -db snp -id {rs_id_str} | efetch -format docsum | xtract -pattern DocumentSummary -element SNP_ID CHRPOS_PREV_ASSM ALLELE SNP_CLASS > eutils_temp1.txt

        with open("eutils_temp1.txt", "rt") as f :

            for line_raw in f.readlines() :
                line = line_raw.strip()

                lineparts = line.split("\t")

                if len(lineparts) == 4 :
                    rs_id, chrom_pos, alt, snp_type = lineparts

                    rs_dict["rs" + str(rs_id)] = [chrom_pos.split(":")[0], chrom_pos.split(":")[1], alt, snp_type]

                else :
                    print("[Warning] Error processing one of the records.")
    
    chroms = []
    poses = []

    for _, row in tissue_df.iterrows() :

        rs_id = row['rs']

        if rs_id in rs_dict :
            chroms.append("chr" + str(rs_dict[rs_id][0]))
            poses.append(int(rs_dict[rs_id][1]))
        else :
            chroms.append("N/A")
            poses.append("N/A")

    tissue_df['chrom'] = chroms
    tissue_df['pos'] = poses
    tissue_df['ref'] = tissue_df['SNP'].apply(lambda x: x.split("_")[-3])
    tissue_df['alt'] = tissue_df['SNP'].apply(lambda x: x.split("_")[-2])
    
    tissue_df.to_csv(tissue_type + ".3aQTL_hg19.csv", sep='\t')

print("Done!")


Processing Adipose_Subcutaneous...
- n_batches = 780
Processing rs record batch 0...
Processing rs record batch 1...
Processing rs record batch 2...
Processing rs record batch 3...
Processing rs record batch 4...
Processing rs record batch 5...
Processing rs record batch 6...
Processing rs record batch 7...
Processing rs record batch 8...
Processing rs record batch 9...
Processing rs record batch 10...
Processing rs record batch 11...
Processing rs record batch 12...
Processing rs record batch 13...
[Warning] Error processing one of the records.
Processing rs record batch 14...
Processing rs record batch 15...
Processing rs record batch 16...
Processing rs record batch 17...
Processing rs record batch 18...
Processing rs record batch 19...
Processing rs record batch 20...
Processing rs record batch 21...
Processing rs record batch 22...
Processing rs record batch 23...
Processing rs record batch 24...
Processing rs record batch 25...
Processing rs record batch 26...
Processing rs recor

Processing rs record batch 218...
Processing rs record batch 219...
Processing rs record batch 220...
Processing rs record batch 221...
Processing rs record batch 222...
Processing rs record batch 223...
Processing rs record batch 224...
Processing rs record batch 225...
Processing rs record batch 226...
Processing rs record batch 227...
Processing rs record batch 228...
Processing rs record batch 229...
Processing rs record batch 230...
Processing rs record batch 231...
Processing rs record batch 232...
Processing rs record batch 233...
Processing rs record batch 234...
Processing rs record batch 235...
Processing rs record batch 236...
Processing rs record batch 237...
Processing rs record batch 238...
Processing rs record batch 239...
Processing rs record batch 240...
Processing rs record batch 241...
Processing rs record batch 242...
Processing rs record batch 243...
Processing rs record batch 244...
Processing rs record batch 245...
Processing rs record batch 246...
Processing rs 

Processing rs record batch 362...
Processing rs record batch 363...
[Warning] Error processing one of the records.
Processing rs record batch 364...
Processing rs record batch 365...
Processing rs record batch 366...
Processing rs record batch 367...
Processing rs record batch 368...
Processing rs record batch 369...
Processing rs record batch 370...
Processing rs record batch 371...
Processing rs record batch 372...
Processing rs record batch 373...
[Warning] Error processing one of the records.
Processing rs record batch 374...
Processing rs record batch 375...
Processing rs record batch 376...
Processing rs record batch 377...
Processing rs record batch 378...
Processing rs record batch 379...
Processing rs record batch 380...
Processing rs record batch 381...
Processing rs record batch 382...
Processing rs record batch 383...
[Warning] Error processing one of the records.
Processing rs record batch 384...
Processing rs record batch 385...
[Warning] Error processing one of the recor

[Warning] Error processing one of the records.
Processing rs record batch 523...
Processing rs record batch 524...
Processing rs record batch 525...
Processing rs record batch 526...
Processing rs record batch 527...
Processing rs record batch 528...
Processing rs record batch 529...
Processing rs record batch 530...
Processing rs record batch 531...
Processing rs record batch 532...
Processing rs record batch 533...
Processing rs record batch 534...
Processing rs record batch 535...
Processing rs record batch 536...
Processing rs record batch 537...
[Warning] Error processing one of the records.
Processing rs record batch 538...
[Warning] Error processing one of the records.
Processing rs record batch 539...
Processing rs record batch 540...
Processing rs record batch 541...
Processing rs record batch 542...
Processing rs record batch 543...
Processing rs record batch 544...
Processing rs record batch 545...
Processing rs record batch 546...
Processing rs record batch 547...
Processin

Processing rs record batch 665...
Processing rs record batch 666...
Processing rs record batch 667...
Processing rs record batch 668...
[Warning] Error processing one of the records.
Processing rs record batch 669...
Processing rs record batch 670...
Processing rs record batch 671...
Processing rs record batch 672...
Processing rs record batch 673...
Processing rs record batch 674...
Processing rs record batch 675...
Processing rs record batch 676...
Processing rs record batch 677...
Processing rs record batch 678...
Processing rs record batch 679...
Processing rs record batch 680...
Processing rs record batch 681...
Processing rs record batch 682...
Processing rs record batch 683...
Processing rs record batch 684...
Processing rs record batch 685...
Processing rs record batch 686...
Processing rs record batch 687...
Processing rs record batch 688...
Processing rs record batch 689...
[Warning] Error processing one of the records.
[Warning] Error processing one of the records.
[Warning]

...

Processing rs record batch 4...
>78 WARNING: >78 FAILURE ( Tue Sep 21 06:04:22 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863686 [RS] OR rs281863676 [RS] OR rs281863661 [RS] OR rs281863522 [RS] OR rs281863401 [RS] OR rs281863349 [RS] OR rs281875005 [RS] OR rs281874939 [RS] OR rs281874818 [RS] OR rs281863219 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<ErrorList>
  <PhraseNotFound>rs281863676[RS]</PhraseNotFound>
  <PhraseNotFound>rs281875005[RS]</PhraseNotFound>
  <PhraseNotFound>rs281863219[RS]</PhraseNotFound>
<ErrorList>>78
SECOND ATTEMPT>78
>78 WARNING: >78 FAILURE ( Tue Sep 21 06:04:25 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863686 [RS] OR rs281863676 [RS] OR rs281863661 [RS] OR rs281863522 [RS] OR rs281863401 [RS] OR rs281863349 [RS] OR rs2818

nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863466 [RS] OR rs281863364 [RS] OR rs281863567 [RS] OR rs281863521 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eSearchResult>
<eSearchResult>
  <Count>1</Count>
  <RetMax>0</RetMax>
  <RetStart>0</RetStart>
  <QueryKey>1</QueryKey>
  <WebEnv>MCID_6149d87ea2a33d50d1733e86</WebEnv>
  <TranslationStack>
    <TermSet>
      <Term>rs281863364[RS]</Term>
      <Field>RS</Field>
      <Count>1</Count>
      <Explode>N</Explode>
    </TermSet>
    <OP>GROUP</OP>
  </TranslationStack>
  <QueryTranslation>rs281863364[RS]</QueryTranslation>
  <ErrorList>
    <PhraseNotFound>rs281863466[RS]</PhraseNotFound>
    <PhraseNotFound>rs281863567[RS]</PhraseNotFound>
    <PhraseNotFound>rs281863521[RS]</PhraseNotFound>
  </ErrorList>
</eSearchResult>>78
QUERY FAILURE>78
[Warning] Error processing 

>78 WARNING: >78 FAILURE ( Tue Sep 21 06:05:41 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863724 [RS] OR rs281863504 [RS] OR rs281863499 [RS] OR rs281863464 [RS] OR rs9274253 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<ErrorList>
  <PhraseNotFound>rs281863724[RS]</PhraseNotFound>
  <PhraseNotFound>rs281863499[RS]</PhraseNotFound>
  <PhraseNotFound>rs281863464[RS]</PhraseNotFound>
<ErrorList>>78
LAST ATTEMPT>78
>78 ERROR: >78 FAILURE ( Tue Sep 21 06:05:43 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863724 [RS] OR rs281863504 [RS] OR rs281863499 [RS] OR rs281863464 [RS] OR rs9274253 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eSearchResult>
<eSearchResult>
  <Count>

Processing rs record batch 2...
[Warning] Error processing one of the records.
[Warning] Error processing one of the records.
Processing rs record batch 3...
Processing rs record batch 4...
[Warning] Error processing one of the records.
Processing rs record batch 5...
[Warning] Error processing one of the records.
[Warning] Error processing one of the records.
Processing rs record batch 6...
>78 WARNING: >78 FAILURE ( Tue Sep 21 06:07:13 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863592 [RS] OR rs281863427 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<ErrorList>
  <PhraseNotFound>rs281863592[RS]</PhraseNotFound>
  <PhraseNotFound>rs281863427[RS]</PhraseNotFound>
<ErrorList>>78
SECOND ATTEMPT>78
>78 WARNING: >78 FAILURE ( Tue Sep 21 06:07:15 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -us

>78 WARNING: >78 FAILURE ( Tue Sep 21 06:12:28 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863504 [RS] OR rs281863499 [RS] OR rs281863464 [RS] OR rs9274253 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<ErrorList>
  <PhraseNotFound>rs281863499[RS]</PhraseNotFound>
  <PhraseNotFound>rs281863464[RS]</PhraseNotFound>
<ErrorList>>78
SECOND ATTEMPT>78
>78 WARNING: >78 FAILURE ( Tue Sep 21 06:12:30 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863504 [RS] OR rs281863499 [RS] OR rs281863464 [RS] OR rs9274253 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<ErrorList>
  <PhraseNotFound>rs281863499[RS]</PhraseNotFound>
  <PhraseNotFound>rs281863464[RS]</PhraseNotFound>
<ErrorList>>78
LAST ATTEMPT>78
>78 ERROR: >78 FAILU

In [7]:

version_suffix = '_v8_lead_w_effect_sizes'


In [9]:
#Compile apaQTL data from GTEx (v8)

def get_snp_pos(row) :
    if row['strand'] == '+' :
        return row['snp_pos'] - row['pas_pos'] + 175 - 1
    else :
        return row['pas_pos'] - row['snp_pos'] + 175

def get_snp_ref(row) :
    if row['strand'] == '+' :
        return row['snp_ref']
    else :
        if row['snp_ref'] == 'A' :
            return 'T'
        if row['snp_ref'] == 'C' :
            return 'G'
        if row['snp_ref'] == 'G' :
            return 'C'
        if row['snp_ref'] == 'T' :
            return 'A'

def get_snp_var(row) :
    if row['strand'] == '+' :
        return row['snp_var']
    else :
        if row['snp_var'] == 'A' :
            return 'T'
        if row['snp_var'] == 'C' :
            return 'G'
        if row['snp_var'] == 'G' :
            return 'C'
        if row['snp_var'] == 'T' :
            return 'A'

def get_var_seq(row) :
    ref_seq = row['wide_seq_ext']
    var_seq = ref_seq[:row['rel_snp_pos']] + row['rel_snp_var'] + ref_seq[row['rel_snp_pos']+1:]
    
    return var_seq

for tissue_type in tissue_types :
    
    print("Processing data for tissue = '" + str(tissue_type) + "'.")
    
    snp_df = pd.read_csv(tissue_type + '.3aQTL_hg19.csv', sep='\t')
    
    snp_df.loc[snp_df['chrom'].isnull(), 'chrom'] = 'Missing'
    snp_df.loc[snp_df['pos'].isnull(), 'pos'] = -1

    snp_df['start'] = snp_df['pos'].astype(int)
    snp_df['end'] = (snp_df['pos'] + 1).astype(int)
    snp_df['gene'] = snp_df['transcript'].apply(lambda x: x.split("@")[1])
    
    snp_df = snp_df.query("pos != -1 and chrom != 'Missing'").copy().reset_index(drop=True)
    
    #Join and collect effect sizes
    effect_size_df = pd.read_csv('with_effect_sizes/' + tissue_type + '.3aQTL.txt', sep='\t')[['SNP', 'gene', 'beta']]
    snp_df['row_id'] = snp_df['SNP'] + "_" + snp_df['transcript'].apply(lambda x: x.split("@")[0])
    effect_size_df['row_id'] = effect_size_df['SNP'] + "_" + effect_size_df['gene'].apply(lambda x: x.split("|")[0])
    
    snp_df = snp_df.join(effect_size_df[['row_id', 'beta']].set_index('row_id'), on='row_id', how='inner').copy().reset_index(drop=True)

    snp_df = snp_df[['chrom', 'start', 'end', 'gene', 'SNP', 'rs', 'transcript', 'p.value', 'beta']]
    
    #Filter on lead SNPs
    snp_df = snp_df.sort_values(by='p.value', ascending=True).drop_duplicates(subset=['transcript'], keep='first').copy().reset_index(drop=True)
    
    snp_df.to_csv(tissue_type + version_suffix + ".3aQTL.coordinates.bed", sep='\t', header=False, index=False)

    !bedtools intersect -a polyadb_coordinates.bed -b {tissue_type + version_suffix}.3aQTL.coordinates.bed -wa -wb > polyadb_{tissue_type + version_suffix}_intersect.bed

    snp_bed_hg19 = pd.read_csv("polyadb_" + tissue_type + version_suffix + "_intersect.bed", sep='\t', error_bad_lines=False, index_col=False, names=['chrom', 'start', 'end', 'gene', 'gene_id', 'strand', 'chr_2', 'snp_pos', 'end_2', 'gene_2', 'snp_id', 'rs_id', 'transcript_id', 'p_val', 'effect_size'])
    
    snp_bed_hg19 = snp_bed_hg19.query("gene == gene_2").copy().reset_index(drop=True)
    
    snp_bed_hg19 = snp_bed_hg19[['chrom', 'start', 'end', 'gene', 'gene_id', 'strand', 'chr_2', 'snp_pos', 'end_2', 'gene_2', 'snp_id', 'transcript_id', 'p_val', 'effect_size']]

    snp_bed_hg19['snp_ref'] = snp_bed_hg19['snp_id'].apply(lambda s: s.split("_")[2])
    snp_bed_hg19['snp_var'] = snp_bed_hg19['snp_id'].apply(lambda s: s.split("_")[3])
    
    polyadb_df_sel = polyadb_df[['wide_seq_ext', 'gene_id', 'pas', 'pas_pos', 'cut_mode', 'site_type', 'rpm', 'total_rpm', 'ratio']].copy().reset_index(drop=True)

    snp_df = snp_bed_hg19.join(polyadb_df_sel.set_index('gene_id'), on='gene_id').copy().reset_index(drop=True)
    
    snp_df['rel_snp_pos'] = snp_df.apply(get_snp_pos, axis=1)
    snp_df['rel_snp_ref'] = snp_df.apply(get_snp_ref, axis=1)
    snp_df['rel_snp_var'] = snp_df.apply(get_snp_var, axis=1)

    snp_df['wide_seq_ext_var'] = snp_df.apply(get_var_seq, axis=1)
    
    #Find transcript ends in hg38
    
    snp_df['transcript_id'] = snp_df['transcript_id'].apply(lambda x: x.split("@")[0])
    snp_df['snp_transcript_id'] = snp_df['snp_id'] + "_" + snp_df['transcript_id']
    
    transcript_end_dict = { row['transcript_id'] : -1 for _, row in snp_df.iterrows() }

    with open("/home/jlinder2/Downloads/GRCh38_latest_genomic.gtf", 'rt') as f :
        for line_raw in f.readlines() :

            a = line_raw.strip()

            if len(a.split("\t")) > 8 :
                start = a.strip().split("\t")[3]
                end = a.strip().split("\t")[4]

                strand = a.strip().split("\t")[6]

                t_id = ""
                if strand in ['+', '-'] :
                    t_id = a.strip().split("\t")[8].split(";")[1].split("\"")[1].split(".")[0]

                if t_id in transcript_end_dict :

                    if transcript_end_dict[t_id] == -1 :
                        if strand == '+' :
                            transcript_end_dict[t_id] = int(end)
                        else :
                            transcript_end_dict[t_id] = int(start)
                    else :
                        if strand == '+' :
                            transcript_end_dict[t_id] = max(transcript_end_dict[t_id], int(end))
                        else :
                            transcript_end_dict[t_id] = min(transcript_end_dict[t_id], int(start))

    qtl_cut_poses = []
    for _, row in snp_df.iterrows() :

        qtl_cut_poses.append(transcript_end_dict[row['transcript_id']])

    snp_df['snp_pos_hg38'] = snp_df['snp_id'].apply(lambda x: int(x.split("_")[1]))
    
    snp_df['qtl_cut_pos'] = qtl_cut_poses
    snp_df['snp_distance'] = np.abs(snp_df['qtl_cut_pos'] - snp_df['snp_pos_hg38'])
    
    #Calculate mean PDUIs
    
    transcript_ids = np.unique(snp_df['transcript_id'].apply(lambda x: x.split("@")[0]).values).tolist()
    transcript_df = pd.DataFrame({'transcript_id' : transcript_ids})
    
    pdui_df = None
    try :
        pdui_df = pd.read_csv(tissue_type + "_combined_All_PDUIs.txt", sep='\t')
    except FileNotFoundError :
        continue
    
    pdui_df['event_id'] = pdui_df['Gene'].apply(lambda x: x.split("|")[0])
    pdui_df = pdui_df.join(transcript_df[['transcript_id']].set_index('transcript_id'), on='event_id', how='inner').copy().reset_index(drop=True)

    gtex_samples = pdui_df.columns.values.tolist()[2:-1]

    def calc_mean_PDUI(row) :

        n_samples = 0.
        PDUI = 0.

        for gtex_sample in gtex_samples :
            
            if not np.isnan(row[gtex_sample]) :

                PDUI += row[gtex_sample]
                n_samples += 1.

        return PDUI / n_samples

    pdui_df['mean_PDUI'] = pdui_df.apply(calc_mean_PDUI, axis=1)
    
    median_mean_PDUI = np.median(pdui_df['mean_PDUI'].values)
    
    tissue_transcript_PDUI_dict = {}
    for _, row in pdui_df.iterrows() :
        tissue_transcript_PDUI_dict[row['event_id']] = row['mean_PDUI']

    PDUIs = []
    for _, row in snp_df.iterrows() :

        mean_PDUI = -1.
        if row['transcript_id'].split("@")[0] in tissue_transcript_PDUI_dict :
            mean_PDUI = tissue_transcript_PDUI_dict[row['transcript_id'].split("@")[0]]
        
        PDUIs.append(mean_PDUI)

    snp_df['mean_PDUI'] = PDUIs
    snp_df['snp_pos'] = snp_df['rel_snp_pos'] - 105
    
    snp_df = snp_df.query("mean_PDUI != -1.").copy().reset_index(drop=True)
    
    snp_df = snp_df.join(polyadb_df[['gene_id', 'sitenum', 'num_sites']].set_index("gene_id"), on='gene_id', how='inner').copy().reset_index(drop=True)
    snp_df['distal_snp'] = (snp_df['num_sites'] - snp_df['sitenum']).apply(lambda x: 1 if x == 0 else -1)
    
    #Drop duplicates
    snp_df['row_id'] = snp_df['gene_id'] + "_" + snp_df['snp_id'] + "_" + snp_df['transcript_id']
    snp_df = snp_df.sort_values(by='snp_id').drop_duplicates(subset=['row_id'], keep='first')

    print(" - len(snp_df) = " + str(len(snp_df)))
    
    #Save final apaQTL dataframe
    snp_df.to_csv("polyadb_" + tissue_type + version_suffix + "_SNPs.csv", sep='\t')


Processing data for tissue = 'Adipose_Subcutaneous'.
 - len(snp_df) = 936
Processing data for tissue = 'Adipose_Visceral_Omentum'.
 - len(snp_df) = 752
Processing data for tissue = 'Adrenal_Gland'.
 - len(snp_df) = 427
Processing data for tissue = 'Artery_Aorta'.
 - len(snp_df) = 594
Processing data for tissue = 'Artery_Coronary'.
 - len(snp_df) = 293
Processing data for tissue = 'Artery_Tibial'.
 - len(snp_df) = 852
Processing data for tissue = 'Brain_Amygdala'.
 - len(snp_df) = 146
Processing data for tissue = 'Brain_Anterior_cingulate_cortex_BA24'.
 - len(snp_df) = 202
Processing data for tissue = 'Brain_Caudate_basal_ganglia'.
 - len(snp_df) = 259
Processing data for tissue = 'Brain_Cerebellar_Hemisphere'.
 - len(snp_df) = 220
Processing data for tissue = 'Brain_Cerebellum'.
 - len(snp_df) = 280
Processing data for tissue = 'Brain_Cortex'.
 - len(snp_df) = 297
Processing data for tissue = 'Brain_Frontal_Cortex_BA9'.
 - len(snp_df) = 243
Processing data for tissue = 'Brain_Hippocamp

In [10]:
#Concatenate dataframes

snp_dfs = []

for tissue_type in tissue_types :
    
    print("Processing tissue = '" + tissue_type + "'")
    
    snp_df = None
    try :
        snp_df = pd.read_csv("polyadb_" + tissue_type + version_suffix + "_SNPs.csv", sep='\t')
    except FileNotFoundError :
        continue
    
    snp_df['tissue_type'] = tissue_type
    
    snp_dfs.append(snp_df)

snp_df = pd.concat(snp_dfs).copy().reset_index(drop=True)


Processing tissue = 'Adipose_Subcutaneous'
Processing tissue = 'Adipose_Visceral_Omentum'
Processing tissue = 'Adrenal_Gland'
Processing tissue = 'Artery_Aorta'
Processing tissue = 'Artery_Coronary'
Processing tissue = 'Artery_Tibial'
Processing tissue = 'Brain_Amygdala'
Processing tissue = 'Brain_Anterior_cingulate_cortex_BA24'
Processing tissue = 'Brain_Caudate_basal_ganglia'
Processing tissue = 'Brain_Cerebellar_Hemisphere'
Processing tissue = 'Brain_Cerebellum'
Processing tissue = 'Brain_Cortex'
Processing tissue = 'Brain_Frontal_Cortex_BA9'
Processing tissue = 'Brain_Hippocampus'
Processing tissue = 'Brain_Hypothalamus'
Processing tissue = 'Brain_Nucleus_accumbens_basal_ganglia'
Processing tissue = 'Brain_Putamen_basal_ganglia'
Processing tissue = 'Brain_Spinal_cord_cervical_c-1'
Processing tissue = 'Brain_Substantia_nigra'
Processing tissue = 'Breast_Mammary_Tissue'
Processing tissue = 'Cells_EBV-transformed_lymphocytes'
Processing tissue = 'Cells_Cultured_fibroblasts'
Processing

In [11]:

merged_df = snp_df


In [12]:

merged_df.to_csv("polyadb_merged" + version_suffix + "_SNPs.csv", sep='\t')


In [13]:
#Compute median effect sizes

exclude_tissues = [['']]
exclude_suffixes = ['']

for exclude_tissue, exclude_suffix in zip(exclude_tissues, exclude_suffixes) :

    print("Excluding " + str(exclude_tissue) + "...")
    
    snp_dict = {}

    for i, tissue_type in enumerate(tissue_types) :
        
        if tissue_type in exclude_tissue :
            continue

        print("Processing tissue = '" + tissue_type + "'")

        snp_df = None
        try :
            snp_df = pd.read_csv("polyadb_" + tissue_type + version_suffix + "_SNPs.csv", sep='\t')
        except FileNotFoundError :
            continue

        for _, row in snp_df.iterrows() :

            snp_id = row['snp_id'] + "__" + row['gene_id'] + "__" + row['transcript_id']

            effect_size = row['effect_size']
            p_val = row['p_val']

            if snp_id not in snp_dict :
                snp_dict[snp_id] = {
                    'effect_size' : [],
                    'p_val' : [],
                    'snp_id' : row['snp_id'],
                    'gene' : row['gene'],
                    'gene_id' : row['gene_id'],
                    'pas' : row['pas'],
                    'wide_seq_ext' : row['wide_seq_ext'],
                    'wide_seq_ext_var' : row['wide_seq_ext_var'],
                    'rel_snp_pos' : row['rel_snp_pos'],
                    'snp_pos' : row['rel_snp_pos'] - 105,
                    'mean_PDUI' : row['mean_PDUI'],
                    'site_type' : row['site_type'],
                    'distal_snp' : row['distal_snp'],
                    'snp_distance' : row['snp_distance'],
                }

            snp_dict[snp_id]['effect_size'].append(effect_size)
            snp_dict[snp_id]['p_val'].append(p_val)

    extended_snp_ids = []
    snp_ids = []
    genes = []
    gene_ids = []
    pases = []
    seqs = []
    seq_vars = []
    rel_snp_poses = []
    snp_poses = []
    effect_sizes = []
    p_vals = []
    n_tissues = []
    mean_PDUIs = []
    site_types = []
    distal_snps = []
    snp_distances = []

    for snp_id in snp_dict :

        extended_snp_ids.append(snp_id)
        snp_ids.append(snp_dict[snp_id]['snp_id'])
        effect_sizes.append(np.median(snp_dict[snp_id]['effect_size']))
        p_vals.append(np.min(snp_dict[snp_id]['p_val']))
        
        genes.append(snp_dict[snp_id]['gene'])
        gene_ids.append(snp_dict[snp_id]['gene_id'])
        pases.append(snp_dict[snp_id]['pas'])
        seqs.append(snp_dict[snp_id]['wide_seq_ext'])
        seq_vars.append(snp_dict[snp_id]['wide_seq_ext_var'])
        rel_snp_poses.append(snp_dict[snp_id]['rel_snp_pos'])
        snp_poses.append(snp_dict[snp_id]['snp_pos'])
        mean_PDUIs.append(snp_dict[snp_id]['mean_PDUI'])
        site_types.append(snp_dict[snp_id]['site_type'])
        distal_snps.append(snp_dict[snp_id]['distal_snp'])
        snp_distances.append(snp_dict[snp_id]['snp_distance'])

        n_tissues.append(len(snp_dict[snp_id]['snp_id']))

    median_df = pd.DataFrame({
        'extended_snp_id' : extended_snp_ids,
        'snp_id' : snp_ids,
        'gene' : genes,
        'gene_id' : gene_ids,
        'site_type' : site_types,
        'pas' : pases,
        'wide_seq_ext' : seqs,
        'wide_seq_ext_var' : seq_vars,
        'rel_snp_pos' : rel_snp_poses,
        'snp_pos' : snp_poses,
        'effect_size' : effect_sizes,
        'mean_PDUI' : mean_PDUIs,
        'p_val' : p_vals,
        'distal_snp' : distal_snps,
        'snp_distance' : snp_distances,
        'n_tissues' : n_tissues
    })
    median_df = median_df[['extended_snp_id', 'snp_id', 'gene', 'gene_id', 'site_type', 'pas', 'wide_seq_ext', 'wide_seq_ext_var', 'rel_snp_pos', 'snp_pos', 'distal_snp', 'snp_distance', 'mean_PDUI', 'p_val', 'effect_size', 'n_tissues']]

    median_df.to_csv("polyadb_median" + version_suffix + exclude_suffix + "_SNPs.csv", sep='\t')


Excluding ['']...
Processing tissue = 'Adipose_Subcutaneous'
Processing tissue = 'Adipose_Visceral_Omentum'
Processing tissue = 'Adrenal_Gland'
Processing tissue = 'Artery_Aorta'
Processing tissue = 'Artery_Coronary'
Processing tissue = 'Artery_Tibial'
Processing tissue = 'Brain_Amygdala'
Processing tissue = 'Brain_Anterior_cingulate_cortex_BA24'
Processing tissue = 'Brain_Caudate_basal_ganglia'
Processing tissue = 'Brain_Cerebellar_Hemisphere'
Processing tissue = 'Brain_Cerebellum'
Processing tissue = 'Brain_Cortex'
Processing tissue = 'Brain_Frontal_Cortex_BA9'
Processing tissue = 'Brain_Hippocampus'
Processing tissue = 'Brain_Hypothalamus'
Processing tissue = 'Brain_Nucleus_accumbens_basal_ganglia'
Processing tissue = 'Brain_Putamen_basal_ganglia'
Processing tissue = 'Brain_Spinal_cord_cervical_c-1'
Processing tissue = 'Brain_Substantia_nigra'
Processing tissue = 'Breast_Mammary_Tissue'
Processing tissue = 'Cells_EBV-transformed_lymphocytes'
Processing tissue = 'Cells_Cultured_fibro

In [14]:
median_df

,extended_snp_id,snp_id,gene,gene_id,site_type,pas,wide_seq_ext,wide_seq_ext_var,rel_snp_pos,snp_pos,distal_snp,snp_distance,mean_PDUI,p_val,effect_size,n_tissues
0,chr10_100986746_C_T_b38__MRPL43.1__NM_032112,chr10_100986746_C_T_b38,MRPL43,MRPL43.1,Intron,-1,GCAAGTCGGTCGAGGAGATCTCGACGCTGGTGCAGAAGCTGGCCGA...,GCAAGTCGGTCGAGGAGATCTCGACGCTGGTGCAGAAGCTGGCCGA...,196,91,-1,477,0.661635,1.429000e-220,-1.140811,23
1,chr10_110292931_C_G_b38__SMNDC1.8__NM_005871,chr10_110292931_C_G_b38,SMNDC1,SMNDC1.8,3_most_exon,0,ATTTTTGTTAATTAGATCAGGTTGTCTGCATTTGTTGGTGTAAGTG...,ATTTTTGTTAATTAGATCAGGTTGTCTGCATTTGTTGGTGTAAGTG...,307,202,-1,2201,0.046592,6.145000e-22,-0.370786,23
2,chr10_110292931_C_G_b38__SMNDC1.9__NM_005871,chr10_110292931_C_G_b38,SMNDC1,SMNDC1.9,3_most_exon,1,GTTTAAGCCAAATACATGCATAGAAAAGGGTCTTCCTATTAATGGA...,GTTTAAGCCAAATACATGCATAGAAAAGGGTCTTCCTATTAATGGA...,234,129,-1,2201,0.046592,6.145000e-22,-0.370786,23
3,chr10_13138287_T_G_b38__OPTN.11__NM_001008212,chr10_13138287_T_G_b38,OPTN,OPTN.11,3_most_exon,2,TTAGTCATATATCTTTCATTAGTTCTATGGATATGAGCAGATCCCT...,TTAGTCATATATCTTTCATTAGTTCTATGGATATGAGCAGATCCCT...,225,120,-1,21,0.251945,2.978000e-80,-0.640202,22
4,chr10_13138287_T_G_b38__OPTN.11__NM_001008213,chr10_13138287_T_G_b38,OPTN,OPTN.11,3_most_exon,2,TTAGTCATATATCTTTCATTAGTTCTATGGATATGAGCAGATCCCT...,TTAGTCATATATCTTTCATTAGTTCTATGGATATGAGCAGATCCCT...,225,120,-1,11,0.254355,1.600000e-77,-0.630945,22
5,chr10_13138287_T_G_b38__OPTN.12__NM_001008212,chr10_13138287_T_G_b38,OPTN,OPTN.12,3_most_exon,1,ACTGGAGCCCAGTATGTGCTGTGTGAGTTAGAAGTCATTCTTGCTG...,ACTGGAGCCCAGTATGTGCTGTGTGAGTTAGAAGTCATTCTTGCTG...,177,72,-1,21,0.251945,2.978000e-80,-0.640202,22
6,chr10_13138287_T_G_b38__OPTN.12__NM_001008213,chr10_13138287_T_G_b38,OPTN,OPTN.12,3_most_exon,1,ACTGGAGCCCAGTATGTGCTGTGTGAGTTAGAAGTCATTCTTGCTG...,ACTGGAGCCCAGTATGTGCTGTGTGAGTTAGAAGTCATTCTTGCTG...,177,72,-1,11,0.254355,1.600000e-77,-0.630945,22
7,chr10_16513432_A_T_b38__PTER.24__NM_001001484,chr10_16513432_A_T_b38,PTER,PTER.24,3_most_exon,0,TGAAATAAAGGGCATTTTGACCTAATACAATTAATTTTCTGGATAA...,TGAAATAAAGGGCATTTTGACCTAATACAATTAATTTTCTGGATAA...,262,157,-1,313,0.805992,1.323000e-55,-0.704307,22
8,chr10_16513432_A_T_b38__PTER.25__NM_001001484,chr10_16513432_A_T_b38,PTER,PTER.25,3_most_exon,0,TGGATAACTCTTAAAGAGAAGTCATTTTAACTGTTTTTGCTACTCC...,TGGATAACTCTTAAAGAGAAGTCATTTTAACTGTTTTTGCTACTCC...,223,118,-1,313,0.805992,1.323000e-55,-0.704307,22
9,chr10_16513432_A_T_b38__PTER.26__NM_001001484,chr10_16513432_A_T_b38,PTER,PTER.26,3_most_exon,0,ATATTGTCATTCAAAATATATTTTAACCCAAAATAAGTTAAATAAT...,ATATTGTCATTCAAAATATATTTTAACCCAAAATAAGTTAAATAAT...,175,70,-1,313,0.805992,1.323000e-55,-0.704307,22


In [15]:

len(median_df['snp_id'].unique())


2714

In [16]:
len(tissue_types)

49

In [17]:

lead_median_df = median_df.copy()

lead_median_df['cut_to_snp'] = np.abs(lead_median_df['snp_pos'] - 100)
lead_median_df = lead_median_df.sort_values(by='cut_to_snp', ascending=True).drop_duplicates(subset=['snp_id'], keep='first')

blacklist_ids = [
    'chr5_76728085_C_T'
]

lead_median_df = lead_median_df.loc[~lead_median_df['snp_id'].isin(blacklist_ids)]

len(lead_median_df.query("snp_pos >= 20 and snp_pos < 126 and pas != -1")['snp_id'].unique())


1253